In [2]:
import sys
sys.path.insert(0, "./lib/")
import WrittingVOC2012TFrecords

import pandas as pd
import tensorflow as tf
import os.path

In [12]:
#This cell reads VOC 2012 dataset and store it in TFrecord objects and save them in a file for future.
#The process of reading data and put them in TFrecord format is time consuming so they are stored in a file.

xml_path = "../VOC2012/Annotations/*.xml"
destination = "../Tfdata/"

#It splits dataset to 90% for training and 10% validation.
if not (os.path.isfile(destination+"test.record") or os.path.isfile(destination+"train.record")):
    WrittingVOC2012TFrecords.run(xml_path, destination)

In [ ]:
# Convolutional Layer 1.# Convo 
filter_size1 = 7          # Convolution filters are 5 x 5 pixels.
num_filters1 = 96         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 7          # Convolution filters are 5 x 5 pixels.
num_filters2 = 265        # There are 36 of these filters.

# Convolutional Layer 3.
filter_size2 = 3          # Convolution filters are 5 x 5 pixels.
num_filters2 = 384        # There are 36 of these filters.

# Convolutional Layer 2.
filter_size2 = 3          # Convolution filters are 5 x 5 pixels.
num_filters2 = 384        # There are 36 of these filters.


# Fully-connected layer.
fc_size = 4096           # Number of neurons in fully-connected layer.
fc_size = 1024           # Number of neurons in fully-connected layer.
fc_size = 1024           # Number of neurons in fully-connected layer.
#fc_size = 9           # Number of neurons in fully-connected layer.

In [ ]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 224

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 3

# Number of classes, one class for each of 10 digits.
num_classes = 9

In [ ]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [ ]:
def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   filter_stride,
                   use_pooling=True,   # Use max-pooling.
                   pooling_size,
                   pooling_stride):  

    # Shape of the filter-weights for the convolution.
    # This format is determined by the TensorFlow API.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Create new weights aka. filters with the given shape.
    weights = new_weights(shape=shape)

    # Create new biases, one for each filter.
    biases = new_biases(length=num_filters)

    # Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1,
    # because the first is for the image-number and
    # the last is for the input-channel.
    # But e.g. strides=[1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[filter_stride, filter_stride, filter_stride, filter_stride],
                         padding='SAME')

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, pooling_size, pooling_size, 1],
                               strides=[1, pooling_stride, pooling_stride, 1],
                               padding='SAME')

    # Rectified Linear Unit (ReLU).
    # It calculates max(x, 0) for each input pixel x.
    # This adds some non-linearity to the formula and allows us
    # to learn more complicated functions.
    layer = tf.nn.relu(layer)

    # Note that ReLU is normally executed before the pooling,
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights

In [ ]:
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

In [ ]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [ ]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   filter_stride = 2,
                   use_pooling=True,   # Use max-pooling.
                   pooling_size = 5,
                   pooling_stride = 2): 

    

link to load data from pascal VOC: https://gist.github.com/saghiralfasly/ee642af0616461145a9a82d7317fb1d6

http://machinelearninguru.com/deep_learning/tensorflow/basics/tfrecord/tfrecord.html